In [316]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [317]:
df = pd.read_csv("../processed-data.csv")
df

,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN
...,...,...,...,...,...,...,...
203,2026-12-01,NaN,NaN,NaN,NaN,NaN,NaN
204,2027-01-01,NaN,NaN,NaN,NaN,NaN,NaN
205,2027-02-01,NaN,NaN,NaN,NaN,NaN,NaN
206,2027-03-01,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
def evaluateModel(df, predictedColumnName):
    validDf = df.dropna(subset=["Value", predictedColumnName])
    
    if len(validDf) == 0:
        return {
            "mse": np.nan,
            "rmse": np.nan,
            "mae": np.nan,
            "mape": np.nan
        }
    
    actual = validDf["Value"]
    predicted = validDf[predictedColumnName]
    
    mse = np.mean((actual - predicted) ** 2)
    
    rmse = np.sqrt(mse)
    
    mae = np.mean(np.abs(actual - predicted))
    
    nonZeroActual = actual[actual != 0]
    nonZeroPredicted = predicted[actual != 0]
    
    if len(nonZeroActual) > 0:
        mape = np.mean(np.abs((nonZeroActual - nonZeroPredicted) / nonZeroActual)) * 100
    else:
        mape = np.nan
    
    return {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "mape": mape
    }

In [319]:
def percentageAverageSeasonalIndex(df):
    df = df.copy()
    df['Month'] = pd.to_datetime(df['Date']).dt.month
    overall_avg = df['Value'].mean()
    df['Percentage'] = df['Value'] / overall_avg * 100
    monthly_index = df.groupby('Month')['Percentage'].mean()
    monthly_index = monthly_index / monthly_index.mean() * 100
    return monthly_index, overall_avg


In [320]:
def predictWithPercentageAverage(df, nFuture=12):
    # work with valid data only for calculations
    validDf = df[df['Value'].notna()].copy()
    idxSeasonal, overallAvg = percentageAverageSeasonalIndex(validDf)
    
    # create extended dataframe starting from original
    extendedDf = df.copy()
    extendedDf['Date'] = pd.to_datetime(extendedDf['Date'])
    
    # add PercAvg_Pred column if it doesn't exist
    if 'PercAvg_Pred' not in extendedDf.columns:
        extendedDf['PercAvg_Pred'] = np.nan
    
    # generate historical predictions for all data points
    for index, row in extendedDf.iterrows():
        if pd.notna(row['Value']):  # only for historical data with values
            month = row['Date'].month
            seasonal = idxSeasonal.loc[month]
            pred = overallAvg * seasonal / 100
            extendedDf.loc[index, 'PercAvg_Pred'] = pred
    
    # get last valid date from original data
    lastValidRow = validDf.iloc[-1]
    lastDate = pd.to_datetime(lastValidRow['Date'])
    
    # generate predictions for future dates (12 months after last valid data)
    for i in range(nFuture):
        futureDate = lastDate + relativedelta(months=i+1)
        predMonth = futureDate.month
        seasonal = idxSeasonal.loc[predMonth]
        pred = overallAvg * seasonal / 100
        
        # check if this date already exists in the dataframe
        existingRow = extendedDf[extendedDf['Date'] == futureDate]
        
        if len(existingRow) > 0:
            # update existing row with prediction
            extendedDf.loc[extendedDf['Date'] == futureDate, 'PercAvg_Pred'] = pred
        else:
            # add new row for this date
            newRow = pd.DataFrame({
                'Date': [futureDate],
                'Value': [np.nan],
                'MovingAvg': [np.nan],
                'PercAvg_Pred': [pred]
            })
            extendedDf = pd.concat([extendedDf, newRow], ignore_index=True)
    
    # sort by date and reset index
    extendedDf = extendedDf.sort_values('Date').reset_index(drop=True)
    
    # convert back to string format
    extendedDf['Date'] = extendedDf['Date'].dt.strftime('%Y-%m-%d')
    
    # display future predictions
    futurePredictions = extendedDf[extendedDf['PercAvg_Pred'].notna() & extendedDf['Value'].isna()]
    print("future predictions:")
    print(futurePredictions[['Date', 'PercAvg_Pred']].to_string())
    
    return extendedDf, idxSeasonal, overallAvg

def execPercentageAverage(nFuture=12):
    global df
    dfLocal = df.copy()
    resultDf, idxSeasonal, overallAvg = predictWithPercentageAverage(dfLocal, nFuture)
    print('indeks musiman bulanan (percentage average):')
    print(idxSeasonal)

    print(f'rata-rata keseluruhan: {overallAvg:.2f}')
    print('\nevaluasi (hanya data historis):')
    print(evaluateModel(resultDf, 'PercAvg_Pred'))
    print('\npreview hasil:')
    print(resultDf.head(20))
    return resultDf

In [321]:
df = execPercentageAverage(nFuture=12)
df

future predictions:
           Date  PercAvg_Pred
184  2025-05-01    409.564129
185  2025-06-01    408.943416
186  2025-07-01    407.124616
187  2025-08-01    405.045130
188  2025-09-01    403.708365
189  2025-10-01    403.983718
190  2025-11-01    405.572502
191  2025-12-01    407.017941
192  2026-01-01    407.037443
193  2026-02-01    407.902507
194  2026-03-01    408.657562
195  2026-04-01    410.147672
indeks musiman bulanan (percentage average):
Month
1      99.994766
2     100.207281
3     100.392772
4     100.758839
5     100.615483
6     100.462996
7     100.016181
8      99.505324
9      99.176928
10     99.244573
11     99.634881
12     99.989975
Name: Percentage, dtype: float64
rata-rata keseluruhan: 407.06

evaluasi (hanya data historis):
{'mse': np.float64(119.84747240364968), 'rmse': np.float64(10.947487036011948), 'mae': np.float64(9.466821931143455), 'mape': np.float64(2.3271976322024654)}

preview hasil:
          Date   Value   MovingAvg  PercAvg_Pred   ExpSmooth     

,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN
...,...,...,...,...,...,...,...
203,2026-12-01,NaN,NaN,NaN,NaN,NaN,NaN
204,2027-01-01,NaN,NaN,NaN,NaN,NaN,NaN
205,2027-02-01,NaN,NaN,NaN,NaN,NaN,NaN
206,2027-03-01,NaN,NaN,NaN,NaN,NaN,NaN


In [322]:
# Export it
outPath = "../processed-data.csv"
df.to_csv(outPath, index=False)
df

,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN
...,...,...,...,...,...,...,...
203,2026-12-01,NaN,NaN,NaN,NaN,NaN,NaN
204,2027-01-01,NaN,NaN,NaN,NaN,NaN,NaN
205,2027-02-01,NaN,NaN,NaN,NaN,NaN,NaN
206,2027-03-01,NaN,NaN,NaN,NaN,NaN,NaN
